In [47]:
# Core Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from itertools import cycle

# Ignore warnings in Python
warnings.filterwarnings('ignore')

In [105]:
import json
import pandas as pd

# List of file paths
file_paths = [
    'C:/Users/jamie/Desktop/PhD Research/Experimental Research Code + Datasets/Experimental Research Code + Datasets/Early Version of Open Seizure Database/Open Seizure Database/osdb_3min_allSeizures.json',
    'C:/Users/jamie/Desktop/PhD Research/Experimental Research Code + Datasets/Experimental Research Code + Datasets/Early Version of Open Seizure Database/Open Seizure Database/osdb_3min_falseAlarms.json',
    'C:/Users/jamie/Desktop/PhD Research/Experimental Research Code + Datasets/Experimental Research Code + Datasets/Early Version of Open Seizure Database/Open Seizure Database/osdb_3min_fallEvents.json'
]

# Initialize an empty list to store data from all files
all_rows = []
current_id = 1  # Start Id counter

# Process each file
for file_path in file_paths:
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Extract data from the JSON file
    for event in data:
        event_id = event.get("id")
        user_id = event.get("userId")
        event_type = event.get("type")
        sub_type = event.get("subType")
        hr1 = event.get('hr')


        # Loop through each datapoint within the event
        for datapoint in event.get("datapoints", []):
            raw_data = None

            # Extract rawData from the nested dataJSON field
            try:
                nested_data = json.loads(datapoint.get("dataJSON", "{}"))
                nested_inner_data = json.loads(nested_data.get("dataJSON", "{}"))
                hr = nested_inner_data.get("hr")
                raw_data = nested_inner_data.get("rawData")
            except json.JSONDecodeError:
                pass

            # Append the extracted data with the current Id
            all_rows.append({
                "Id": current_id,
                "eventId": event_id,
                "userId": user_id,
                "type": event_type,
                "subType": sub_type,
                "hr": hr,
                "rawData": raw_data
            })
            current_id += 1  # Increment Id

# Create a single DataFrame
df = pd.DataFrame(all_rows)

# Display the DataFrame
df.tail()

,Id,eventId,userId,type,subType,hr,rawData
9291,9292,14898,156,Fall,Uncontrolled,0,"[1004.324016377599, 1005.6113176704044, 1008.3..."
9292,9293,14898,156,Fall,Uncontrolled,0,"[1000.1011495342927, 1004.0482531052368, 1000...."
9293,9294,14898,156,Fall,Uncontrolled,0,"[1001.1701269235376, 1002.7462560430771, 998.6..."
9294,9295,14898,156,Fall,Uncontrolled,0,"[1012.4248508971073, 1004.5334132492027, 1003...."
9295,9296,14898,156,Fall,Uncontrolled,0,"[1004.9762153019517, 1000.7225907604744, 1005...."


In [107]:
# Define the subType values to drop
subtypes_to_drop = [None, 'null', '', 'Unknown']

# Identify eventIds that have these subType values
event_ids_to_drop = df[df['subType'].isin(subtypes_to_drop)]['eventId'].unique()

# Filter out rows with these eventIds
df_filtered = df[~df['eventId'].isin(event_ids_to_drop)]

# Rename 'Uncontrolled' to 'Fall'
#df_filtered['subType'] = df_filtered['subType'].replace('Uncontrolled', 'Fall')
df_filtered.loc[df_filtered['subType'] == 'Other', 'subType'] = 'Other_Seizure'
df_filtered.loc[df_filtered['subType'] == 'Uncontrolled', 'subType'] = 'Fall'
df_filtered.loc[df_filtered['subType'] == 'Other (Please describe in notes)', 'subType'] = 'Other_False_Alarm'

# Replace spaces with underscores in the 'subType' column
df_filtered.loc[:, 'subType'] = df_filtered['subType'].str.replace(' ', '_')

# Print all unique subType values after replacement
unique_subtypes = df_filtered['subType'].unique()
print("Unique subTypes after filtering, renaming, and replacing spaces:", unique_subtypes)


Unique subTypes after filtering, renaming, and replacing spaces: ['Other_Seizure' 'Tonic-Clonic' 'Aura' 'Typing' 'Pushing_Pram/Wheelchair'
 'Washing_/_cleaning' 'Other_False_Alarm' 'Talking' 'Brushing_Teeth'
 'Sorting' 'Motor_Vehicle' 'Cycling' 'Computer_Games' 'Fall']
